In [4]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [19]:
face_classifier=cv2.CascadeClassifier('C:\\Users\\sahay\\OneDrive\\Desktop\\opencv\data\\haarcascades\\haarcascade_frontalface_default.xml')
emotion_model = load_model('C:\\Users\\sahay\\OneDrive\\Desktop\\Final_Assignment\\facial_emotion\\emotion_detection.h5')
age_model = load_model('C:\\Users\\sahay\\OneDrive\\Desktop\\Final_Assignment\\age_gender\\age_model.h5')
gender_model = load_model('C:\\Users\\sahay\\OneDrive\\Desktop\\Final_Assignment\\age_gender\\gender_model.h5')
configPath ='C:\\Users\\sahay\\OneDrive\\Desktop\\Final_Assignment\\object_detection\\ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'C:\\Users\\sahay\\OneDrive\\Desktop\\Final_Assignment\\object_detection\\frozen_inference_graph.pb' 
object_model = cv2.dnn_DetectionModel(weightsPath,configPath)
object_model.setInputSize(320,320)
object_model.setInputScale(1.0/ 127.5)
object_model.setInputMean ((127.5, 127.5, 127.5))
object_model.setInputSwapRB (True)

<dnn_Model 00000226B93158B0>

In [20]:
class_labels=['Angry','Disgust', 'Fear', 'Happy','Neutral','Sad','Surprise']
gender_labels = ['Male', 'Female']
objct_class_names= ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 
                     'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 
                     'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 
                     'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 
                     'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 
                     'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 
                     'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 
                     'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 
                     'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 
                     'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 
                     'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 
                     'hair drier', 'toothbrush', 'hair brush']

In [21]:
# press q to exit from the real time live stream window

In [23]:
cap=cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)

while True:
    ret,frame=cap.read()
    labels=[]
    
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces=face_classifier.detectMultiScale(gray,1.3,5)
    classIds, confs, bbox = object_model.detect(frame,confThreshold=0.5) 
    
    if cv2.waitKey(1) & 0xFF == ord('o'):
        if len(classIds) != 0:
            
            #object detection
           for classId, confidence, box in zip(classIds.flatten(),confs.flatten(), bbox):
              cv2.rectangle(frame,box,color=(0,255,0), thickness=2)
              cv2.putText(frame,objct_class_names[classId-1].upper(),(box[0]+10, box[1]+30),
                          cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    else:
        for (x,y,w,h) in faces:
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
            roi_gray=gray[y:y+h,x:x+w]
            roi_gray=cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

            #Get image ready for prediction
            roi=roi_gray.astype('float')/255  
            roi=img_to_array(roi)
            roi=np.expand_dims(roi,axis=0)

            #Facial Emotion
            preds=emotion_model.predict(roi)[0]  
            label=class_labels[preds.argmax()]  
            label_position=(x+h,y+25)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

            #Gender
            roi_color=frame[y:y+h,x:x+w]
            roi_color=cv2.resize(roi_color,(200,200),interpolation=cv2.INTER_AREA)
            gender_predict = gender_model.predict(np.array(roi_color).reshape(-1,200,200,3))
            gender_predict = (gender_predict>= 0.5).astype(int)[:,0]
            gender_label=gender_labels[gender_predict[0]] 
            gender_label_position=(x+h,y+55) 
            cv2.putText(frame,gender_label,gender_label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)

            #Age
            age_predict = age_model.predict(np.array(roi_color).reshape(-1,200,200,3))
            age = round(age_predict[0,0])
            age_label_position=(x+h,y+85)
            cv2.putText(frame,str(age),age_label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
        
   
    cv2.imshow('Object,Age,Gender and Emotion Detector', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()